In [1]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import tensorflow as tf
import numpy as np
import pickle as pkl
from sklearn import preprocessing as pre
#from pathos.pools import ProcessPool as Pool
tf.set_random_seed(4321)
np.random.seed(4321)

#from xmlrpc import client
#import xmlrpclib
NEWDS=1
#s = xmlrpclib.ServerProxy('http://10.21.230.64:8778')

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [3]:
#some hypers
GO='<START>'
STOP='<END>'
PAD='<PAD>'
unknown='<UNKNOWN>'
BATCH=128
BEAM_WIDTH=9
EPOCHS=100
LAM=0.9
embedding_size=512
lstm_units=2048
dropout_keep_prob=0.8
PATIENCE=30
PATIENCE_MONITOR=True
GLOVE=False #if true embedding size will reset to 300
CONSTGLOVE=False
MAX_LEN=33
#new

SIN=True
NUM_BLOCKS=3
NUM_HEADS = 8
FEAT_DIM=2048
NUM_UNITS=512
FRAMES=28
WARMUP=4000
TRUE_BATCH=1024
GRAD_APPLY=TRUE_BATCH/BATCH
LR=0.0001#*GRAD_APPLY

In [4]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model
if GLOVE:
    embedding_size=300

In [5]:
#load data
with open('./crt_split/token_train.pkl') as f:
    trainCaptions=pkl.load(f)
with open('./crt_split/token_dev.pkl') as f:
    devCaptions=pkl.load(f)
with open('./crt_split/token_test.pkl') as f:
    testCaptions=pkl.load(f)

In [6]:
def processToken(caps):
    nk=[]
    
    for i in caps:
        t=[GO]+list(i[1])+[STOP]
        empty=33-len(t)
        
        #t=t+[pad]*empty
        nk.append([i[0],t])
    return nk

In [7]:
trainCaptions=processToken(trainCaptions)
devCaptions=processToken(devCaptions)
testCaptions=processToken(testCaptions)

In [8]:
def listofwords(data):
    '''takes a list of sentences nd returns vocab'''
    a=[]
    for i in data:
        for j in i[1]:
            if j not in a:
                a.append(j)
    return a

In [9]:
#find the vocab and size
trainVocab=listofwords(trainCaptions+devCaptions)
trainVocabSize=len(trainVocab)

In [10]:
#find cap len
trainCapLen=[len(i[1]) for i in trainCaptions]
devCapLen=[len(i[1]) for i in devCaptions]
testCapLen=[len(i[1]) for i in testCaptions]

In [11]:
#Label Encoder for output transform
pre_op=pre.LabelEncoder()
pre_op.fit(trainVocab+[PAD])
onehoter=np.identity(len(pre_op.classes_))

In [12]:
#word to int
trainSeq=[pre_op.transform(i[1]) for i in trainCaptions]
devSeq=[pre_op.transform(i[1]) for i in devCaptions]
trainID=[i[0] for i in trainCaptions]
devID=[i[0] for i in devCaptions]



#word to int with pool
p=Pool(8)


trainSeq=p.map(pre_op.transform,[i[1] for i in trainCaptions])
devSeq=p.map(pre_op.transform,[i[1] for i in devCaptions])
trainID=[i[0] for i in trainCaptions]
devID=[i[0] for i in devCaptions]



In [13]:
#appending stops


trainSeqReg=[np.pad(i,(0,MAX_LEN-len(i)),'constant',constant_values=pre_op.transform([PAD])) for i in trainSeq]
devSeqReg=[np.pad(i,(0,MAX_LEN-len(i)),'constant',constant_values=pre_op.transform([PAD])) for i in devSeq]
trainSeqReg=np.array(trainSeqReg)
devSeqReg=np.array(devSeqReg)

In [14]:
#loading video features
videoFeats=np.load(file='./crt_split/consilidated_feats.npy')
videoFeatSize=np.array([len(i) for i in videoFeats])
#making the shape regular
videoFeats=np.array([np.pad(i,mode='constant',pad_width=[(0,28-len(i)),(0,0)]) for i in videoFeats])

In [15]:
#glove
if GLOVE:
    gloveModel=loadGloveModel('./glove/glove.6B.300d.txt')
    gloveEmbedding=[]
    for i in pre_op.classes_:
        if gloveModel.has_key(i):
            gloveEmbedding.append(gloveModel[i])
        else:
            gloveEmbedding.append(np.random.normal(size=(300)))
    gloveEmbedding=np.array(gloveEmbedding)

In [16]:
from nltk.translate.bleu_score import corpus_bleu
def calBleu(ref,cap):
    score_4 = corpus_bleu(ref,cap,weights=(0.25,0.25,0.25,0.25))
    return score_4


In [17]:
from modules import *



In [18]:
#Lets build the graph
tf.reset_default_graph()


In [19]:
padID=pre_op.transform([PAD])[0]
startID=pre_op.transform([GO])[0]
stopID=pre_op.transform([STOP])[0]


In [20]:
#dummys
enc_ph = tf.placeholder(tf.float32,(BATCH,FRAMES,2048))
enc_len_ph = tf.placeholder(tf.float32,(BATCH))
'''
y_ph = tf.placeholder('int32',(BATCH,None))
y_len_ph=tf.placeholder('int32',(BATCH))
miniB=tf.placeholder('int32')
'''

"\ny_ph = tf.placeholder('int32',(BATCH,None))\ny_len_ph=tf.placeholder('int32',(BATCH))\nminiB=tf.placeholder('int32')\n"

In [21]:
source_seq = tf.placeholder(shape=(None,28,2048),dtype=tf.float32)
target_seq = tf.placeholder(shape=(None,33),dtype=tf.int32)
source_seq_len = tf.placeholder(shape=(None,), dtype=tf.int32)
target_seq_len = tf.placeholder(shape=(None,), dtype=tf.int32)
no_start_target_seq = tf.placeholder(shape=(None,32),dtype=tf.int32)
batch_size = tf.placeholder(shape=(None),dtype=tf.int32)
real_target_seq_len= tf.placeholder(shape=(None,), dtype=tf.int32)
keep_prob= tf.placeholder(dtype=tf.float32)
end_sentence_emb= tf.placeholder(shape=(None,1024),dtype=tf.float32)

In [22]:
#class Transformer:
'''
    xs: tuple of
        x: int32 tensor. (N, T1)
        x_seqlens: int32 tensor. (N,)
        sents1: str tensor. (N,)
    ys: tuple of
        decoder_input: int32 tensor. (N, T2)
        y: int32 tensor. (N, T2)
        y_seqlen: int32 tensor. (N, )
        sents2: str tensor. (N,)
    training: boolean.
    
def __init__(self, hp):
    #self.hp = hp
    #self.token2idx, self.idx2token = load_vocab(hp.vocab)
'''
embeddings = get_token_embeddings(trainVocabSize, embedding_size, zero_pad=True)

def encode(xs, training=True):
    '''
    Returns
    memory: encoder outputs. (N, T1, d_model)
    '''
    #return xs[0]
    with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):
        x, seqlens  = xs

        # embedding
        enc = x # (N, T1, d_model)
        enc *= FEAT_DIM**0.5 # scale

        enc += positional_encoding(enc, FRAMES)
        enc = tf.layers.dropout(enc, 1-dropout_keep_prob, training=training)

        ## Blocks
        for i in range(NUM_BLOCKS):
            with tf.variable_scope("num_blocks_{}".format(i), reuse=tf.AUTO_REUSE):
                # self-attention
                enc = multihead_attention(queries=enc,
                                          keys=enc,
                                          values=enc,
                                          num_heads=NUM_HEADS,
                                          dropout_rate=1-dropout_keep_prob,
                                          training=training,
                                          causality=False)
                # feed forward
                enc = ff(enc, num_units=[FEAT_DIM, FEAT_DIM])
    memory = enc
    return memory



In [23]:
m=encode((enc_ph,enc_len_ph))

In [24]:
def decode(ys, memory, training=True):
    '''
    memory: encoder outputs. (N, T1, d_model)
    Returns
    logits: (N, T2, V). float32.
    y_hat: (N, T2). int32
    y: (N, T2). int32
    sents2: (N,). string.
    '''
    if training:
        keep_prob=dropout_keep_prob
    else:
        keep_prob=1.0
    #embeddings
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        embedding_matrix_decode = tf.get_variable(
        name="embedding_matrix_de",
        shape=[trainVocabSize, embedding_size],
        dtype=tf.float32)
        decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_decode, target_seq) 

    
    #memory=tf.random_normal(dtype=tf.float32,shape=(BATCH,32,2048))
    encoder_outputs=memory
    enc_mean=tf.reduce_mean(memory,axis=1)
    decoder_initial_state=tf.contrib.rnn.LSTMStateTuple(enc_mean,enc_mean)
    
    #Projection layer and decoder cell
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        output_layer = tf.layers.Dense(trainVocabSize)

        decoder_cell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_units),input_keep_prob=keep_prob,
                                              output_keep_prob=keep_prob)

    
    #attention
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        attention_mechanism_train = tf.contrib.seq2seq.LuongAttention(lstm_units,encoder_outputs)
    
    #attention
    #expri
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(
            encoder_outputs, multiplier=BEAM_WIDTH)

        attention_mechanism_infer = tf.contrib.seq2seq.LuongAttention(lstm_units,tiled_encoder_outputs)
    
    #decoder Attention wrapper
    #expri
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        decoder_cell_train = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism_train,
                attention_layer_size=lstm_units,alignment_history=False)
        decoder_initial_state_train = decoder_cell_train.zero_state(BATCH, tf.float32).clone(cell_state=decoder_initial_state)
    

        #Training helper and decoder
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        helper = tf.contrib.seq2seq.TrainingHelper(decoder_input_embedded,target_seq_len)
        decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell_train, helper, initial_state=decoder_initial_state_train,output_layer=output_layer)#,output_layer=projection_layer)
        outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder)
        logits = outputs.rnn_output
        sample_ids = outputs.sample_id
    '''
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=no_start_target_seq,logits=logits)

    target_weights = tf.sequence_mask(real_target_seq_len, target_seq_len[0], dtype=logits.dtype)

    loss1=tf.reduce_mean(cross_entropy*target_weights)
    '''
    y_ = label_smoothing(tf.one_hot(no_start_target_seq, depth=trainVocabSize))
    ce = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_)
    nonpadding = tf.to_float(tf.not_equal(no_start_target_seq,padID ))  # 0: <pad>
    loss1 = tf.reduce_sum(ce * nonpadding) / (tf.reduce_sum(nonpadding) + 1e-7)
    
    tiled_decoder_initial_state=tf.contrib.seq2seq.tile_batch(decoder_initial_state, multiplier=BEAM_WIDTH)
    #inference
    
    
    


    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        decoder_cell_infer = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism_infer,
                attention_layer_size=lstm_units,alignment_history=False)
        decoder_initial_state_infer = decoder_cell_infer.zero_state(BATCH*BEAM_WIDTH, tf.float32).clone(cell_state=tiled_decoder_initial_state)
    
    decoder_initial_state_infer = decoder_cell_infer.zero_state(BATCH*BEAM_WIDTH, tf.float32).clone(cell_state=tiled_decoder_initial_state)
        #Beam Search decoder
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):

        decoder_initial_state_tiled=decoder_cell_infer.zero_state(batch_size=BATCH*BEAM_WIDTH,dtype=tf.float32).clone(cell_state=tiled_decoder_initial_state)


        # Define a beam-search decoder
        decoder3 = tf.contrib.seq2seq.BeamSearchDecoder(
                cell=decoder_cell_infer,
                embedding=embedding_matrix_decode,
                start_tokens=tf.fill([BATCH],np.int32(pre_op.transform([GO])[0])),
                end_token=np.int32(pre_op.transform([STOP])[0]),
                initial_state=decoder_initial_state_tiled,
                beam_width=BEAM_WIDTH,
                output_layer=output_layer,
                length_penalty_weight=0.0)
        outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder3,maximum_iterations=32+10)


        trs_beam=outputs.predicted_ids
    
    '''
    #expri
    #decoder Attention wrapper
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        decoder_cell_infer = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism_infer,
                attention_layer_size=lstm_units,alignment_history=False)
        decoder_initial_state_infer = decoder_cell_infer.zero_state(BATCH*BEAM_WIDTH, tf.float32).clone(cell_state=tiled_decoder_initial_state)
    
    
    #Beam Search decoder
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        decoder_initial_state_tiled = tf.contrib.seq2seq.tile_batch(
            decoder_initial_state_infer[0], multiplier=BEAM_WIDTH)

        decoder_initial_state_tiled=decoder_cell_infer.zero_state(batch_size=BATCH*BEAM_WIDTH,dtype=tf.float32).clone(cell_state=decoder_initial_state_tiled)


        # Define a beam-search decoder
        decoder3 = tf.contrib.seq2seq.BeamSearchDecoder(
                cell=decoder_cell_infer,
                embedding=embedding_matrix_decode,
                start_tokens=tf.fill([batch_size],np.int32(pre_op.transform([GO])[0])),
                end_token=np.int32(pre_op.transform([STOP])[0]),
                initial_state=decoder_initial_state_tiled,
                beam_width=BEAM_WIDTH,
                output_layer=output_layer,
                length_penalty_weight=0.0)
        outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder3,maximum_iterations=32+10)


        trs_beam=outputs.predicted_ids
    '''
    
    return logits, loss1 , trs_beam,logits

In [25]:
t=decode(0,m)

def decode(ys, memory, training=True):
    '''
    memory: encoder outputs. (N, T1, d_model)
    Returns
    logits: (N, T2, V). float32.
    y_hat: (N, T2). int32
    y: (N, T2). int32
    sents2: (N,). string.
    '''
    if training:
        keep_prob=dropout_keep_prob
    else:
        keep_prob=1.0
    #embeddings
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        embedding_matrix_decode = tf.get_variable(
        name="embedding_matrix_de",
        shape=[trainVocabSize, embedding_size],
        dtype=tf.float32)
    decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_decode, ys[0]) 
    
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
    #attention
    
        attention_mechanism_train = tf.contrib.seq2seq.LuongAttention(lstm_units,memory)
    
    
    #attention
    #expri
    
        tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(memory, multiplier=BEAM_WIDTH)

        attention_mechanism_infer = tf.contrib.seq2seq.LuongAttention(lstm_units,tiled_encoder_outputs)
    
    #print tiled_encoder_outputs,attention_mechanism_infer
    
    #projection layer & decoder cell
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        output_layer = tf.layers.Dense(trainVocabSize)

        decoder_cell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_units),input_keep_prob=keep_prob,
                                              output_keep_prob=keep_prob)
    mean_mem=tf.reduce_mean(memory,axis=1)
    decoder_initial_state=tf.contrib.rnn.LSTMStateTuple(mean_mem,mean_mem)
    
    tiled_decoder_initial_state=tf.contrib.seq2seq.tile_batch(decoder_initial_state, multiplier=BEAM_WIDTH)
    
    decoder_initial_state=tf.contrib.rnn.LSTMStateTuple(tf.random_normal((BATCH,2048),stddev=0.1),tf.random_normal((BATCH,2048),stddev=0.1))
    #print tiled_decoder_initial_state
    #decoder Attention wrapper
    #expri
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        decoder_cell_train = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism_train,
                attention_layer_size=lstm_units,alignment_history=False)
        decoder_initial_state_train = decoder_cell_train.zero_state(BATCH, tf.float32).clone(cell_state=decoder_initial_state)
    
    #Training helper and decoder
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        helper = tf.contrib.seq2seq.TrainingHelper(decoder_input_embedded,ys[2])
        decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell_train, helper, initial_state=decoder_initial_state_train,output_layer=output_layer)#,output_layer=projection_layer)
        outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=33)
        logits = outputs.rnn_output
        sample_ids = outputs.sample_id
    
    '''
    
    #expri
#decoder Attention wrapper
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        decoder_cell_infer = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism_infer,
                attention_layer_size=lstm_units,alignment_history=False)
        
        decoder_initial_state_infer = decoder_cell_infer.zero_state(BATCH*BEAM_WIDTH, tf.float32).clone(cell_state=tiled_decoder_initial_state)
    
    
    
    
        #Beam Search decoder
    with tf.variable_scope("myScope",reuse=tf.AUTO_REUSE):
        
        decoder_initial_state_tiled=decoder_cell_infer.zero_state(batch_size=BATCH*BEAM_WIDTH,dtype=tf.float32).clone(cell_state=tiled_decoder_initial_state)

    
        # Define a beam-search decoder
        decoder3 = tf.contrib.seq2seq.BeamSearchDecoder(
                cell=decoder_cell_infer,
                embedding=embedding_matrix_decode,
                start_tokens=tf.fill([BATCH],np.int32(pre_op.transform([GO])[0])),
                end_token=np.int32(pre_op.transform([STOP])[0]),
                initial_state=decoder_initial_state_tiled,
                beam_width=BEAM_WIDTH,
                output_layer=output_layer,
                length_penalty_weight=0.0)
        outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder3,maximum_iterations=33)


        trs_beam=outputs.predicted_ids
    '''
    
    #loss1
    #tmp=tf.zeros((128, 32, 10329))
    #tmp[:,:,:]=logits
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=ys[1],logits=logits)
    #print cross_entropy.shape[1]
    target_weights = tf.sequence_mask(ys[2], miniB, dtype=logits.dtype)

    loss1=tf.reduce_mean(cross_entropy*target_weights)


    return logits, loss1 , target_weights,logits

def decode(ys, memory, training=True):
    '''
    memory: encoder outputs. (N, T1, d_model)
    Returns
    logits: (N, T2, V). float32.
    y_hat: (N, T2). int32
    y: (N, T2). int32
    sents2: (N,). string.
    '''
    with tf.variable_scope("decoder", reuse=tf.AUTO_REUSE):
        decoder_inputs, y, seqlens = ys

        # embedding
        dec = tf.nn.embedding_lookup(embeddings, decoder_inputs)  # (N, T2, d_model)
        dec *= NUM_UNITS ** 0.5  # scale

        dec += positional_encoding(dec, MAX_LEN-1)
        dec = tf.layers.dropout(dec, 1-dropout_keep_prob, training=training)

        # Blocks
        for i in range(NUM_BLOCKS):
            with tf.variable_scope("num_blocks_{}".format(i), reuse=tf.AUTO_REUSE):
                # Masked self-attention (Note that causality is True at this time)
                dec = multihead_attention(queries=dec,
                                          keys=dec,
                                          values=dec,
                                          num_heads=NUM_HEADS,
                                          dropout_rate=1-dropout_keep_prob,
                                          training=training,
                                          causality=True,
                                          scope="self_attention")

                # Vanilla attention
                dec = multihead_attention(queries=dec,
                                          keys=memory,
                                          values=memory,
                                          num_heads=NUM_HEADS,
                                          dropout_rate=1-dropout_keep_prob,
                                          training=training,
                                          causality=False,
                                          scope="vanilla_attention")
                ### Feed Forward
                dec = ff(dec, num_units=[NUM_UNITS*4,embedding_size])

    # Final linear projection (embedding weights are shared)
    weights = tf.transpose(embeddings) # (d_model, vocab_size)
    logits = tf.einsum('ntd,dk->ntk', dec, weights) # (N, T2, vocab_size)
    y_hat = tf.to_int32(tf.argmax(logits, axis=-1))

    return logits, y_hat, y

In [26]:

def train( xs, ys):
    '''
    Returns
    loss: scalar.
    train_op: training operation
    global_step: scalar.
    summaries: training summary node
    '''
    # forward
    memory = encode(xs)
    logits, loss, preds,sample_ids= decode(ys,memory)

    # train scheme
    '''
    y_ = label_smoothing(tf.one_hot(y, depth=trainVocabSize))
    ce = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_)
    nonpadding = tf.to_float(tf.not_equal(y,padID ))  # 0: <pad>
    loss = tf.reduce_sum(ce * nonpadding) / (tf.reduce_sum(nonpadding) + 1e-7)
    
    varss   = tf.trainable_variables() 
    lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in varss if 'bias' not in v.name ]) * 0.001
    '''
    
    global_step = tf.train.get_or_create_global_step()
    lr = noam_scheme(LR, global_step, WARMUP)
    optimizer = tf.train.AdamOptimizer(lr)
    train_op = optimizer.minimize(loss, global_step=global_step)
    '''
    #https://stackoverflow.com/questions/45987156/tensorflow-average-gradients-over-several-batches 
    t_vars = tf.trainable_variables()
    # create a copy of all trainable variables with `0` as initial values
    accum_tvars = [tf.Variable(tf.zeros_like(t_var.initialized_value()),trainable=False) for t_var in t_vars]                                        
    # create a op to initialize all accums vars
    zero_ops = [tv.assign(tf.zeros_like(tv)) for tv in accum_tvars]

    # compute gradients for a batch
    batch_grads_vars = optimizer.compute_gradients(loss, t_vars)
    # collect the batch gradient into accumulated vars
    accum_ops = [accum_tvars[i].assign_add(batch_grad_var[0]) for i, batch_grad_var in enumerate(batch_grads_vars)]

    # apply accums gradients 
    train_op = optimizer.apply_gradients([(accum_tvars[i], batch_grad_var[1]) for i, batch_grad_var in enumerate(batch_grads_vars)])
    # train_step = opt.apply_gradients(zip(accum_tvars, zip(*batch_grads_vars)[1])
    
    optimizer = tf.train.AdamOptimizer()
    gradients, variables = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 10.0)
    train_op = optimizer.apply_gradients(zip(gradients, variables))
    '''
    tf.summary.scalar('lr', lr)
    tf.summary.scalar("loss", loss)
    #tf.summary.scalar("global_step", global_step)
    #global_step = tf.train.get_or_create_global_step()
    summaries = tf.summary.merge_all()
    
    
    return loss, train_op,logits, global_step, summaries,loss#gradients#,accum_ops,zero_ops


In [27]:

def evalr(xs, ys):
    '''Predicts autoregressively
    At inference, input ys is ignored.
    Returns
    y_hat: (N, T2)
    
    decoder_inputs, y, y_seqlen = ys

    decoder_inputs = tf.ones((tf.shape(xs[0])[0], 1), tf.int32) * startID
    ys = (decoder_inputs, y, y_seqlen)
    '''
    memory = encode(xs, False)
    logits, loss, preds,extra= decode(ys, memory,False)

    return preds#, summaries


In [28]:
loss, train_op ,logits,global_step, summaries,extra= train((enc_ph,enc_len_ph),0)#(y_ph[:,:-1],y_ph[:,1:],y_len_ph))
y_hat = evalr((enc_ph,enc_len_ph),0)#(y_ph[:,:-1],y_ph[:,1:],y_len_ph))

In [29]:
#makes training batch

SenEmb=np.squeeze(np.load('./LmEmb.npy'))

def getTrainBatch(indexs):
    sourceBatch=np.array([videoFeats[trainID[i]-NEWDS] for i in indexs])
    targetBatch=np.array([trainSeqReg[i] for i in indexs])
    sourceBatchLen=np.array([videoFeatSize[trainID[i]-NEWDS] for i in indexs])
    targetBatchLen=np.array([trainCapLen[i] for i in indexs])
    targetSenEm=np.array([SenEmb[i] for i in indexs])
    return sourceBatch,targetBatch,sourceBatchLen,targetBatchLen,targetSenEm

#makes dev batch
def getDevBatch(indexs):
    sourceBatch=np.array([videoFeats[devID[i]-NEWDS] for i in indexs])
    targetBatch=np.array([devSeqReg[i] for i in indexs])
    sourceBatchLen=np.array([videoFeatSize[devID[i]-NEWDS] for i in indexs])
    targetBatchLen=np.array([devCapLen[i] for i in indexs])
    return sourceBatch,targetBatch,sourceBatchLen,targetBatchLen

def calValBleu():
    data=videoFeats[1200:1300]
    data_len=videoFeatSize[1200:1300]
    if BATCH>100:
        data=np.concatenate([data,videoFeats[:BATCH-100]])
        data_len=np.concatenate([data_len,videoFeatSize[:BATCH-100]])
    gen_sum=[]
    for i in range(len(data)/BATCH):
        start=i*BATCH
        stop=(i+1)*BATCH

        
        y=sess.run(y_hat,feed_dict={enc_ph:data[start:stop],y_ph:np.zeros((BATCH,MAX_LEN)),
                                              enc_len_ph:data_len[start:stop],
                                                y_len_ph:np.zeros((BATCH))})
        
        
        for t in y:
            gen_sum.append(t)

    if BATCH<100:        
        start=len(data)-BATCH
        stop=len(data)
        y=sess.run(y_hat,feed_dict={enc_ph:data[start:stop],y_ph:np.zeros((BATCH,MAX_LEN)),
                                              enc_len_ph:data_len[start:stop],
                                                y_len_ph:np.zeros((BATCH))})
                        
        

        y=y[-(len(data)-len(gen_sum)):]
        for t in y:
            gen_sum.append(t)
    
    gen_sum=gen_sum[:100]
    #processing summaries
    summs=[]
    for i in gen_sum:
        summ=''
        for j in i:
            if j==stopID:
                break
            summ = summ+' '+pre_op.inverse_transform(j)
        summs.append(summ[1:])
    vdo=1200
    with open('gen_dev.txt','w+') as fle:
        for i in summs:
            fle.write('beam_size_5'+'\tvid'+str(vdo)+'\t'+i)
            fle.write('\n')
            vdo+=1
    with open('gen_dev.txt','r') as fle:
        pred=fle.read()
    return s.calcScore(pred)

In [30]:
def calValBleu():
    data=videoFeats[1200:1300]
    data_len=videoFeatSize[1200:1300]
    if BATCH>100:
        data=np.concatenate([data,videoFeats[:BATCH-100]])
        data_len=np.concatenate([data_len,videoFeatSize[:BATCH-100]])
    gen_sum=[]
    for i in range(len(data)/BATCH):
        start=i*BATCH
        stop=(i+1)*BATCH

        
        y=sess.run(y_hat,feed_dict={enc_ph:data[start:stop],
                                              enc_len_ph:data_len[start:stop],
                                                })
        
        y=y[:,:,0]
        for t in y:
            gen_sum.append(t)

    if BATCH<100:        
        start=len(data)-BATCH
        stop=len(data)
        y=sess.run(y_hat,feed_dict={enc_ph:data[start:stop],
                                              enc_len_ph:data_len[start:stop],
                                                })
                        
        
        y=y[:,:,0]
        y=y[-(len(data)-len(gen_sum)):]
        for t in y:
            gen_sum.append(t)
    
    gen_sum=gen_sum[:100]
    
    #print gen_sum.shape
    #processing summaries
    summs=[]
    for i in gen_sum:
        summ=''
        for j in i:
            if j==stopID:
                break
            summ = summ+' '+pre_op.inverse_transform(j)
        summs.append(summ[1:])
    
    
    ref=[]
    for i in range(100):
        ref.append([])
    for i in devCaptions:
        ref[i[0]-1200-NEWDS].append(i[1][1:-1])

    cap=[]
    for i in summs:
        cap.append(i.split())

    score=calBleu(ref,cap)
    
    
    return score

In [31]:
saver=tf.train.Saver()

In [32]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

In [33]:
warnings.filterwarnings('ignore')

In [34]:
summary_writer = tf.summary.FileWriter('./log/', sess.graph)

In [35]:
#dont touch
maxtlen=max(trainCapLen)
maxvlen=max(devCapLen)
t_newlen=[maxtlen-1 for i in range(len(trainCapLen))]
v_newlen=[maxtlen-1 for i in range(len(devCapLen))]

In [36]:
#training starts here
bestVal=0
patience=PATIENCE

print 'starting training'
training_losses=[]
valid_losses=[]
valid_bleu=[]
tData=np.arange(len(trainSeqReg))
dData=np.arange(len(devSeqReg))
steps=0
#sess.run(zero_ops)
for j in range(EPOCHS):
    np.random.shuffle(tData) #makes them iid
    training_loss=0
    for i in range(len(trainSeqReg)/BATCH):
        start=i*BATCH
        stop=(i+1)*BATCH
        sourceBatch,targetBatch,sourceBatchLen,targetBatchLen,senTargetBatch=getTrainBatch(tData[start:stop])
        
        '''
        _,lost,_s,_gs=sess.run([accum_ops,loss,summaries,global_step],feed_dict={enc_ph:sourceBatch,y_ph:targetBatch,enc_len_ph:sourceBatchLen,
                                                y_len_ph:targetBatchLen})
        
        summary_writer.add_summary(_s, _gs)
        steps+=1
        
        if steps%GRAD_APPLY==0:
            sess.run(train_op)
            sess.run(zero_ops)
        '''
        maxpossible=max(targetBatchLen)+1
        '''
        print maxpossible
        
        
        l=sess.run(extra[1],feed_dict={enc_ph:sourceBatch,enc_len_ph:sourceBatchLen,
                                                target_seq:targetBatch,
                                                target_seq_len:t_newlen[start:stop],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:BATCH,keep_prob:dropout_keep_prob,
                                                end_sentence_emb:senTargetBatch
                                                })
        print l[0]
        '''
        #break
        #print targetBatch[0],targetBatchLen[0],sourceBatch[0],sourceBatchLen[0]
        '''
        if maxpossible>=34:
            continue
        _,lost,_s,_gs=sess.run([train_op,loss,summaries,global_step],feed_dict={enc_ph:sourceBatch,y_ph:targetBatch[:,:maxpossible],enc_len_ph:sourceBatchLen,
                                                y_len_ph:targetBatchLen,miniB:maxpossible-1})
        print lost
        '''
        
        _,lost,_s,_gs=sess.run([train_op,loss,summaries,global_step],feed_dict={enc_ph:sourceBatch,enc_len_ph:sourceBatchLen,
                                                target_seq:targetBatch,
                                                target_seq_len:t_newlen[start:stop],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:BATCH,keep_prob:dropout_keep_prob,
                                                end_sentence_emb:senTargetBatch
                                                })
        
        summary_writer.add_summary(_s, _gs)
        training_loss+=lost
        #print lost
    #calculate t_loss
    training_losses.append(training_loss/(len(trainSeqReg)/BATCH))
    
    #calculate v_loss
    #disabled
    '''
    validation_loss=0
    for k in range(len(devSeqReg)/BATCH):
        start=k*BATCH
        stop=(k+1)*BATCH
        sourceBatch,targetBatch,sourceBatchLen,targetBatchLen=getDevBatch(dData[start:stop])
        lost=sess.run(loss,feed_dict={enc_ph:sourceBatch,y_ph:targetBatch,enc_len_ph:sourceBatchLen,
                                                y_len_ph:targetBatchLen})
        validation_loss += lost
    
    
    valid_losses.append(validation_loss/len(devSeqReg))
    '''
    
    valBleu=calValBleu()
    valid_bleu.append(valBleu)
    
    
    if(valid_bleu[-1]>bestVal) and PATIENCE_MONITOR==True:
        bestVal=valid_bleu[-1]
        saver.save(sess, "transModels/best.ckpt")
        print "saving model best"
        patience=PATIENCE
    
    print "Epoch:%d training loss:%.4f: valid loss:%.4f valid bleu:%.4f"% (j,training_losses[-1],0.0,valid_bleu[-1])
    patience-=1
    if patience==0 and PATIENCE_MONITOR==True:
        break
    

starting training
saving model best
Epoch:0 training loss:6.3838: valid loss:0.0000 valid bleu:0.0000
saving model best
Epoch:1 training loss:4.9238: valid loss:0.0000 valid bleu:0.4358
saving model best
Epoch:2 training loss:4.3477: valid loss:0.0000 valid bleu:0.5187
saving model best
Epoch:3 training loss:3.8294: valid loss:0.0000 valid bleu:0.5276
saving model best
Epoch:4 training loss:3.5164: valid loss:0.0000 valid bleu:0.5655
Epoch:5 training loss:3.3515: valid loss:0.0000 valid bleu:0.4960
saving model best
Epoch:6 training loss:3.2490: valid loss:0.0000 valid bleu:0.5876
Epoch:7 training loss:3.1675: valid loss:0.0000 valid bleu:0.5444
Epoch:8 training loss:3.0987: valid loss:0.0000 valid bleu:0.5073
Epoch:9 training loss:3.0403: valid loss:0.0000 valid bleu:0.5052
Epoch:10 training loss:2.9839: valid loss:0.0000 valid bleu:0.5121
Epoch:11 training loss:2.9165: valid loss:0.0000 valid bleu:0.5040
Epoch:12 training loss:2.8518: valid loss:0.0000 valid bleu:0.5451
Epoch:13 trai

#restore model
saver.restore(sess, "BestModel/model.ckpt")

In [37]:
#save model
saver.save(sess, "BestTrans/resume.ckpt")

'BestTrans/resume.ckpt'

In [38]:
if PATIENCE_MONITOR:
    saver.restore(sess, "transModels/best.ckpt")

INFO:tensorflow:Restoring parameters from transModels/best.ckpt


In [39]:
valid_bleu

[8.225113061299963e-78,
 0.4358031727409411,
 0.5187017703902669,
 0.5276458224667989,
 0.5655438635216826,
 0.4959940418096737,
 0.5876046610067263,
 0.5443763615326478,
 0.5073216606659753,
 0.5052007952978532,
 0.5121034220380604,
 0.5039886410536626,
 0.5450657639451099,
 0.5690041109923824,
 0.5457212175281723,
 0.5739125050114697,
 0.5627802793830623,
 0.5883805068342941,
 0.5577031339207043,
 0.5610012311959837,
 0.5357527352256197,
 0.546266224333698,
 0.5438831750862665,
 0.5696619614539277,
 0.528504846958879,
 0.5635960145221541,
 0.5492538476826754,
 0.579128717035524,
 0.5377479106636786,
 0.5786135121756347,
 0.5478406472213724,
 0.5341032008606288,
 0.5265546960467371,
 0.5739277503125324,
 0.5502817942794727,
 0.5454871635500864,
 0.5409306001138258,
 0.5195234725282566,
 0.5456664508540165,
 0.5447908990188346,
 0.5526759359218706,
 0.533608104082931,
 0.5459429128814148,
 0.5254939039775095,
 0.5606402398592442,
 0.5330164101149911,
 0.5330147582908316]

In [40]:
data=videoFeats[1300:]
data_len=videoFeatSize[1300:]
if BATCH==1024:
    data=np.concatenate([data,data[:354]])
    data_len=np.concatenate([data_len,data_len[:354]])
gen_sum=[]
for i in range(len(data)/BATCH):
    start=i*BATCH
    stop=(i+1)*BATCH
    
    y=sess.run(y_hat,feed_dict={enc_ph:data[start:stop],enc_len_ph:data_len[start:stop],
                                              batch_size:BATCH,keep_prob:1.0})
        
    y=y[:,:,0]
    for t in y:
        gen_sum.append(t)

if BATCH<1024:        
    start=len(data)-BATCH
    stop=len(data)
    y=sess.run(y_hat,feed_dict={enc_ph:data[start:stop],enc_len_ph:data_len[start:stop],
                                              batch_size:BATCH,keep_prob:1.0})
    
    y=y[:,:,0]
    y=y[-(len(data)-len(gen_sum)):]
    for t in y:
        gen_sum.append(t)


In [41]:
gen_sum=gen_sum[:670]

data=videoFeats[1300:1364]
data_len=videoFeatSize[1300:1364]
gen_sum=[]
for i in range(len(data)/BATCH):
    start=i*BATCH
    stop=(i+1)*BATCH
    load_trs=trs
    y=sess.run(load_trs,feed_dict={source_seq:data[start:stop],
                                               source_seq_len:data_len[start:stop],
                                              batch_size:BATCH,
                                                keep_prob:1.0
                                                })
    for t in y:
        gen_sum.append(t)


In [42]:
#processing summaries
summs=[]
for i in gen_sum:
    summ=''
    for j in i:
        if j==stopID:
            break
        summ = summ+' '+pre_op.inverse_transform(j)
    summs.append(summ[1:])

for i in range(len(summs)):
    print i,summs[i],vops[i]

vops=np.load('../MSVD/clip_index/testIndex.npy')[:64]
ref_test

#real shit
with open('ref_dev.txt','w+') as fle:
    for i in devCaptions:
        fle.write('vid'+str(i[0])+'\t'+' '.join(i[1][1:-1]))
        fle.write('\n')

In [43]:
vdo=1301
with open('genTrans_LSTM_newDS.txt','w+') as fle:
    for i in summs:
        fle.write('beam_size_1'+'\tvid'+str(vdo)+'\t'+i+'.')
        fle.write('\n')
        vdo+=1

In [44]:
37013

37013

In [45]:

ref=[]
for i in range(670):
    ref.append([])
for i in testCaptions:
    ref[i[0]-1300-NEWDS].append(i[1][1:-1])

cap=[]
for i in summs:
    cap.append(i.split())
    
calBleu(ref,cap)

0.5261747834362036

In [46]:
calValBleu()

0.5883805068342941

In [47]:
summary_writer.close()

In [48]:
trainVocabSize

10209